In [1]:
import pandas as pd
import os
import re

In [8]:
# Ruta del archivo Excel
file_path = 'data/original/Ubicación actual de las Versiones Restauradas.xlsx'

# Especifica el nombre o índice de la hoja que deseas importar
sheet_name = 'VERSIONES RESTAURADAS'  # También puedes usar el índice de la hoja, por ejemplo: 0

# Cargar la hoja especificada en un DataFrame
df_verificacion = pd.read_excel(file_path, sheet_name=sheet_name)

# Mostrar las primeras filas del DataFrame
print(df_verificacion.head())


              Nombre de la serie  \
0                  ¿QUÉ HORA ES?   
1         A Orillas Del Amazonas   
2              Ángulos De Bogotá   
3  Batalla Del Pantano De Vargas   
4     Cali, la Sultana del Valle   

                       Código versión restaurada        LTO  \
0                   VR F35mm 851242 QUÉ HORA ES?  Pendiente   
1         VR F35mm 851304 A Orillas Del Amazonas   000124L8   
2              VR F35mm 851303 Ángulos De Bogotá  Pendiente   
3  VR F35mm 851011 Batalla del pantano de Vargas   000124L8   
4     VR F35mm 851112 Cali, la Sultana del Valle   000173L8   

  VERSIONES RESTAURADAS\nSENALMEMORIA-10: 10.10.113.61  \
0                                                 Sí     
1                                                 Sí     
2                                                 Sí     
3                                                 No     
4                                                 No     

            OBSERVACIONES DE VERIFICACIÓN EN LOS NAS  \

In [9]:
df_codigos = pd.DataFrame()

df_codigos[['Codigo', 'Resto']] = df_verificacion['Código versión restaurada'].str.extract(r'(^.*\d{6})\s*(.*)')

# Mostrar el nuevo DataFrame con las dos columnas
print(df_codigos[['Codigo', 'Resto']])

               Codigo                                              Resto
0     VR F35mm 851242                                       QUÉ HORA ES?
1     VR F35mm 851304                             A Orillas Del Amazonas
2     VR F35mm 851303                                  Ángulos De Bogotá
3     VR F35mm 851011                      Batalla del pantano de Vargas
4     VR F35mm 851112                         Cali, la Sultana del Valle
...               ...                                                ...
1324  VR F16mm 802705                   Yuruparí Cap Cumbia sobre el río
1325  VR F16mm 802706               Yuruparí Cap Las farotas de Talaigua
1326  VR F16mm 802707  Yuruparí Cap Mompox, el ocaso del oro y del barro
1327  VR F16mm 802708               Yuruparí Cap El Cristo negro de Tadó
1328  VR F16mm 802709                Yuruparí Cap El Carnaval del Diablo

[1329 rows x 2 columns]


In [4]:
# Función para limpiar el código
def clean_code(code):
    return str(code).upper().strip()  # Convierte a mayúsculas y elimina espacios

# Función para preprocesar los archivos CSV
def preprocess_csv_files(folder_path):
    preprocessed_data = []
    # Iterar sobre los archivos en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            # Cargar solo la primera columna (asumiendo que es la columna con los códigos)
            df = pd.read_csv(file_path, usecols=[0], header=None)  # Cargar solo la primera columna
            df['cleaned_code'] = df[0].apply(clean_code)  # Limpiar los códigos
            preprocessed_data.append((filename, df[['cleaned_code']]))  # Guardar nombre de archivo y DataFrame limpio
    return preprocessed_data

# Función para crear un diccionario con los códigos
def create_code_map(preprocessed_data):
    code_map = {}
    for filename, df in preprocessed_data:
        for idx, row in df.iterrows():
            code = row['cleaned_code']
            if code not in code_map:
                code_map[code] = []  # Inicializa una lista para múltiples ocurrencias
            code_map[code].append((filename, idx))  # Guarda el archivo y la fila
    return code_map

# Función para verificar la existencia de un código
def verificar_existencia(codigo):
    cleaned_codigo = clean_code(codigo)  # Limpiar el código a buscar
    if cleaned_codigo in code_map:  # Verificar si está en el diccionario
        return "si", code_map[cleaned_codigo][0][0], code_map[cleaned_codigo][0][1]  # Retorna "si", el archivo y la fila
    return "no", None, None  # Retorna "no" si no se encuentra

# Ejecutar el preprocesamiento y la creación del diccionario
folder_path = 'data/original/nas_csv'  # Ruta donde están los archivos CSV
preprocessed_data = preprocess_csv_files(folder_path)  # Preprocesar los archivos CSV
code_map = create_code_map(preprocessed_data)  # Crear el diccionario de códigos

# Supongamos que df_codigos es tu DataFrame con los códigos a verificar
df_codigos = pd.DataFrame({'Codigo': ['VR F35MM 851242', 'UMT 205827 CLIP 1', 'DV 270979']})

# Aplicar la función para verificar existencia
df_codigos[['existe', 'archivo', 'fila']] = df_codigos['Codigo'].apply(verificar_existencia).apply(pd.Series)

# Mostrar el DataFrame actualizado
print(df_codigos)


              Codigo existe    archivo  fila
0    VR F35MM 851242     no       None   NaN
1  UMT 205827 CLIP 1     si  nas_2.csv   0.0
2          DV 270979     si  nas_2.csv   2.0


In [10]:
# Función para limpiar el código
def clean_code(code):
    return str(code).upper().strip()  # Convierte a mayúsculas y elimina espacios

# Función para preprocesar los archivos CSV
def preprocess_csv_files(folder_path):
    preprocessed_data = []
    # Iterar sobre los archivos en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            # Cargar solo las dos primeras columnas
            df = pd.read_csv(file_path, usecols=[0, 1], header=None)  # Cargar solo la primera y segunda columna
            df.columns = ['code', 'extension']  # Asignar nombres a las columnas
            df['cleaned_code'] = df['code'].apply(clean_code)  # Limpiar los códigos
            preprocessed_data.append((filename, df[['cleaned_code', 'extension']]))  # Guardar nombre de archivo y DataFrame limpio
    return preprocessed_data

# Función para crear un diccionario con los códigos
def create_code_map(preprocessed_data):
    code_map = {}
    for filename, df in preprocessed_data:
        for idx, row in df.iterrows():
            code = row['cleaned_code']
            if code not in code_map:
                code_map[code] = []  # Inicializa una lista para múltiples ocurrencias
            code_map[code].append((filename, idx, row['extension']))  # Guarda el archivo, la fila y la extensión
    return code_map

# Función para verificar la existencia de un código
def verificar_existencia(codigo):
    cleaned_codigo = clean_code(codigo)  # Limpiar el código a buscar
    if cleaned_codigo in code_map:  # Verificar si está en el diccionario
        return "si", code_map[cleaned_codigo][0][0], code_map[cleaned_codigo][0][1], code_map[cleaned_codigo][0][2]  # Retorna "si", el archivo, la fila y la extensión
    return "no", None, None, None  # Retorna "no" si no se encuentra

# Ejecutar el preprocesamiento y la creación del diccionario
folder_path = 'data/original/nas_csv'  # Ruta donde están los archivos CSV
preprocessed_data = preprocess_csv_files(folder_path)  # Preprocesar los archivos CSV
code_map = create_code_map(preprocessed_data)  # Crear el diccionario de códigos

# Supongamos que df_codigos es tu DataFrame con los códigos a verificar
df_codigos = pd.DataFrame({'Codigo': ['VR F35MM 851242', 'UMT 205827 CLIP 1', 'DV 270979']})

# Aplicar la función para verificar existencia
df_codigos[['existe', 'archivo', 'fila', 'extension']] = df_codigos['Codigo'].apply(verificar_existencia).apply(pd.Series)

# Mostrar el DataFrame actualizado
print(df_codigos)


              Codigo existe    archivo  fila extension
0    VR F35MM 851242     no       None   NaN      None
1  UMT 205827 CLIP 1     si  nas_2.csv   0.0       mov
2          DV 270979     si  nas_2.csv   2.0       mov


In [11]:
import os
import pandas as pd

# Función para limpiar el código
def clean_code(code):
    return str(code).upper().strip()  # Convierte a mayúsculas y elimina espacios

# Función para preprocesar los archivos CSV
def preprocess_csv_files(folder_path):
    preprocessed_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            # Cargar solo las dos primeras columnas
            df = pd.read_csv(file_path, usecols=[0, 1], header=None)  # Cargar solo la primera y segunda columna
            df.columns = ['code', 'extension']  # Asignar nombres a las columnas
            df['cleaned_code'] = df['code'].apply(clean_code)  # Limpiar los códigos
            preprocessed_data.append((filename, df[['cleaned_code', 'extension']]))  # Guardar nombre de archivo y DataFrame limpio
    return preprocessed_data

# Función para crear un diccionario con los códigos
def create_code_map(preprocessed_data):
    code_map = {}
    for filename, df in preprocessed_data:
        for idx, row in df.iterrows():
            code = row['cleaned_code']
            if code not in code_map:
                code_map[code] = []  # Inicializa una lista para múltiples ocurrencias
            code_map[code].append((filename, idx, row['extension']))  # Guarda el archivo, la fila y la extensión
    return code_map

# Función para verificar la existencia de un código
def verificar_existencia(codigo):
    cleaned_codigo = clean_code(codigo)  # Limpiar el código a buscar
    if cleaned_codigo in code_map:  # Verificar si está en el diccionario
        return "si", code_map[cleaned_codigo][0][0], code_map[cleaned_codigo][0][1], code_map[cleaned_codigo][0][2]  # Retorna "si", el archivo, la fila y la extensión
    return "no", None, None, None  # Retorna "no" si no se encuentra

# Ejecutar el preprocesamiento y la creación del diccionario
folder_path = 'data/original/nas_csv'  # Ruta donde están los archivos CSV
preprocessed_data = preprocess_csv_files(folder_path)  # Preprocesar los archivos CSV
code_map = create_code_map(preprocessed_data)  # Crear el diccionario de códigos

# Supongamos que df_codigos es tu DataFrame con los códigos a verificar
df_codigos = pd.DataFrame({'Codigo': ['VR F35MM 851242', 'UMT 205827 CLIP 1', 'DV 270979', 'DV 277625', 'UMT 216433', 'C1P 242909']})

# Aplicar la función para verificar existencia
df_codigos[['existe', 'archivo', 'fila', 'extension']] = df_codigos['Codigo'].apply(verificar_existencia).apply(pd.Series)

# Mostrar el DataFrame actualizado
print(df_codigos)


              Codigo existe    archivo  fila extension
0    VR F35MM 851242     no       None   NaN      None
1  UMT 205827 CLIP 1     si  nas_2.csv   0.0       mov
2          DV 270979     si  nas_2.csv   2.0       mov
3          DV 277625     si  nas_2.csv   7.0       mov
4         UMT 216433     si  nas_2.csv   8.0       mov
5         C1P 242909     si  nas_2.csv   6.0     mp4  


In [12]:
import os
import pandas as pd

# Función para limpiar el código
def clean_code(code):
    return str(code).upper().strip()  # Convierte a mayúsculas y elimina espacios

# Función para preprocesar los archivos CSV
def preprocess_csv_files(folder_path):
    preprocessed_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            # Cargar solo las dos primeras columnas
            df = pd.read_csv(file_path, usecols=[0, 1], header=None)  # Cargar solo la primera y segunda columna
            df.columns = ['code', 'extension']  # Asignar nombres a las columnas
            df['cleaned_code'] = df['code'].apply(clean_code)  # Limpiar los códigos
            preprocessed_data.append((filename, df[['cleaned_code', 'extension']]))  # Guardar nombre de archivo y DataFrame limpio
    return preprocessed_data

# Función para verificar la existencia de un código
def verificar_existencia(codigo):
    cleaned_codigo = clean_code(codigo)  # Limpiar el código a buscar
    for key in code_map.keys():  # Iterar a través de los códigos preprocesados
        if cleaned_codigo in key:  # Verificar si el código a buscar está en el código
            return "si", code_map[key][0][0], code_map[key][0][1], code_map[key][0][2]  # Retorna "si", el archivo, la fila y la extensión
    return "no", None, None, None  # Retorna "no" si no se encuentra

# Ejecutar el preprocesamiento y la creación del diccionario
folder_path = 'data/original/nas_csv'  # Ruta donde están los archivos CSV
preprocessed_data = preprocess_csv_files(folder_path)  # Preprocesar los archivos CSV

# Crear un diccionario con las claves como los códigos preprocesados
code_map = {}
for filename, df in preprocessed_data:
    for idx, row in df.iterrows():
        code = row['cleaned_code']
        if code not in code_map:
            code_map[code] = []  # Inicializa una lista para múltiples ocurrencias
        code_map[code].append((filename, idx, row['extension']))  # Guarda el archivo, la fila y la extensión

# Supongamos que df_codigos es tu DataFrame con los códigos a verificar
df_codigos = pd.DataFrame({'Codigo': ['VR F35MM 851242', 'UMT 205827 CLIP 1', 'DV 270979', 'DV 277625', 'UMT 216433', 'C1P 242909']})

# Aplicar la función para verificar existencia
df_codigos[['existe', 'archivo', 'fila', 'extension']] = df_codigos['Codigo'].apply(verificar_existencia).apply(pd.Series)

# Mostrar el DataFrame actualizado
print(df_codigos)


              Codigo existe     archivo  fila extension
0    VR F35MM 851242     si  nas_10.csv   291       mov
1  UMT 205827 CLIP 1     si   nas_2.csv     0       mov
2          DV 270979     si   nas_2.csv     2       mov
3          DV 277625     si   nas_2.csv     7       mov
4         UMT 216433     si   nas_2.csv     8       mov
5         C1P 242909     si   nas_2.csv     6     mp4  


In [13]:
import os
import pandas as pd
import re

# Función para preprocesar los archivos CSV
def preprocess_csv_files(folder_path):
    preprocessed_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            # Cargar solo las dos primeras columnas
            df = pd.read_csv(file_path, usecols=[0, 1], header=None)  # Cargar solo la primera y segunda columna
            df.columns = ['code', 'extension']  # Asignar nombres a las columnas
            preprocessed_data.append((filename, df))  # Guardar nombre de archivo y DataFrame sin limpiar
    return preprocessed_data

# Función para verificar la existencia de un código
def verificar_existencia(codigo):
    for key in code_map.keys():  # Iterar a través de los códigos preprocesados
        # Crear una expresión regular para buscar el código en la cadena
        pattern = re.escape(codigo)  # Escapar caracteres especiales en el código
        if re.search(pattern, key, re.IGNORECASE):  # Ignorar mayúsculas/minúsculas
            return "si", code_map[key][0][0], code_map[key][0][1], code_map[key][0][2]  # Retorna "si", el archivo, la fila y la extensión
    return "no", None, None, None  # Retorna "no" si no se encuentra

# Ejecutar el preprocesamiento y la creación del diccionario
folder_path = 'data/original/nas_csv'  # Ruta donde están los archivos CSV
preprocessed_data = preprocess_csv_files(folder_path)  # Preprocesar los archivos CSV

# Crear un diccionario con las claves como los códigos preprocesados
code_map = {}
for filename, df in preprocessed_data:
    for idx, row in df.iterrows():
        code = row['code']
        if code not in code_map:
            code_map[code] = []  # Inicializa una lista para múltiples ocurrencias
        code_map[code].append((filename, idx, row['extension']))  # Guarda el archivo, la fila y la extensión

# Supongamos que df_codigos es tu DataFrame con los códigos a verificar
df_codigos = pd.DataFrame({'Codigo': ['VR F35MM 851242', 'UMT 205827 CLIP 1', 'DV 270979', 'DV 277625', 'UMT 216433', 'C1P 242909']})

# Aplicar la función para verificar existencia
df_codigos[['existe', 'archivo', 'fila', 'extension']] = df_codigos['Codigo'].apply(verificar_existencia).apply(pd.Series)

# Mostrar el DataFrame actualizado
print(df_codigos)


              Codigo existe     archivo  fila extension
0    VR F35MM 851242     si  nas_10.csv   291       mov
1  UMT 205827 CLIP 1     si   nas_2.csv     0       mov
2          DV 270979     si   nas_2.csv     2       mov
3          DV 277625     si   nas_2.csv     7       mov
4         UMT 216433     si   nas_2.csv     8       mov
5         C1P 242909     si   nas_2.csv     6     mp4  
